# Example - using earthkit.data to read from FDB

In [1]:
import os
import earthkit.data

Example environment variables which need to be set:

In [2]:
# os.environ['FDB_HOME'] = '/scratch/mch/vcherkas/spack-root/linux-sles15-zen3/gcc-11.3.0/fdb-5.11.17-7nmpdzsfv62vwoi3u3m7zd7jo556jeyt'
# os.environ['FDB5_HOME'] = os.environ['FDB_HOME']
# os.environ['FDB5_CONFIG'] = "{'type':'local','engine':'toc','schema':'/scratch/mch/vcherkas/fdb-setup/mars/fdb-schema','spaces':[{'handler':'Default','roots':[{'path':'/opr/vcherkas/fdb_root'}]}]}"
# os.environ['ECCODES_DEFINITION_PATH'] = '/scratch/mch/vcherkas/eccodes-cosmo-resources/definitions:/scratch/mch/vcherkas/eccodes/definitions'

In [3]:
class MissingEnvironmentVariable(Exception):
    pass

if os.getenv('FDB_HOME') is None:
    raise MissingEnvironmentVariable('FDB_HOME needs to be set (for pyfdb). Find with `spack location -i fdb`.')
    
if os.getenv('FDB5_HOME') is None:
    raise MissingEnvironmentVariable('FDB5_HOME needs to be set (for earthkit.data). Should be identical to FDB_HOME.')
    
if (os.getenv('FDB5_CONFIG') is None and os.getenv('FDB5_CONFIG_FILE') is None):
    raise MissingEnvironmentVariable('Either FDB5_CONFIG or FDB5_CONFIG_FILE needs to be set (for FDB).')
    
if os.getenv('ECCODES_DEFINITION_PATH') is None:
    raise MissingEnvironmentVariable('ECCODES_DEFINITION_PATH needs to be set (for reading COSMO data)')

We can use pyFDB.list to send incomplete requests, to identify which data is available in FDB.

In [4]:
import pyfdb

request = {
    "date":"20230201",
    "time":"0300",
    "class":"od",
    "stream":"enfo",
    "type":"ememb",
    "model":"COSMO-1E",
    "expver":"0001",
    "step":["1"],
    "number":["1"],
    "levtype":"pl",
    "param":"500014"
    }


levellist = set()
for el in pyfdb.list(request, True, True):
    levellist.add(el['keys']['levelist'])
    
print(levellist)

{'450', '350', '700', '550', '800', '200', '300', '150', '400', '850', '925', '650', '900', '950', '500', '750', '100', '600', '975', '1000', '250'}


And then send a full request with all the mars params of the schema, to fetch the data.

In [5]:
request = {
    "date":"20230201",
    "time":"0300",
    "class":"od",
    "stream":"enfo",
    "type":"ememb",
    "model":"COSMO-1E",
    "expver":"0001",
    "step":["1"],
    "number":["1"],
    "levtype":"pl",
    "levelist":list(levellist),
    "param":"500014"
    }

# When we use batch_size=0 all the fields are loaded into memory and the resulting 
# object will behave like a FieldList:

ds = earthkit.data.from_source("fdb", request, batch_size=0)
ds

In [6]:
ds.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,lssw,T,isobaricInhPa,450,20230201,300,1,cp,1,rotated_ll
1,lssw,T,isobaricInhPa,350,20230201,300,1,cp,1,rotated_ll
2,lssw,T,isobaricInhPa,700,20230201,300,1,cp,1,rotated_ll
3,lssw,T,isobaricInhPa,550,20230201,300,1,cp,1,rotated_ll
4,lssw,T,isobaricInhPa,800,20230201,300,1,cp,1,rotated_ll
5,lssw,T,isobaricInhPa,200,20230201,300,1,cp,1,rotated_ll
6,lssw,T,isobaricInhPa,300,20230201,300,1,cp,1,rotated_ll
7,lssw,T,isobaricInhPa,150,20230201,300,1,cp,1,rotated_ll
8,lssw,T,isobaricInhPa,400,20230201,300,1,cp,1,rotated_ll
9,lssw,T,isobaricInhPa,850,20230201,300,1,cp,1,rotated_ll


In [7]:
for f in ds:
    print(f)

GribField(T,450,20230201,300,1,1)
GribField(T,350,20230201,300,1,1)
GribField(T,700,20230201,300,1,1)
GribField(T,550,20230201,300,1,1)
GribField(T,800,20230201,300,1,1)
GribField(T,200,20230201,300,1,1)
GribField(T,300,20230201,300,1,1)
GribField(T,150,20230201,300,1,1)
GribField(T,400,20230201,300,1,1)
GribField(T,850,20230201,300,1,1)
GribField(T,925,20230201,300,1,1)
GribField(T,650,20230201,300,1,1)
GribField(T,900,20230201,300,1,1)
GribField(T,950,20230201,300,1,1)
GribField(T,500,20230201,300,1,1)
GribField(T,750,20230201,300,1,1)
GribField(T,100,20230201,300,1,1)
GribField(T,600,20230201,300,1,1)
GribField(T,975,20230201,300,1,1)
GribField(T,1000,20230201,300,1,1)
GribField(T,250,20230201,300,1,1)


In [8]:
ds.sel(levelist=950).ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,lssw,T,isobaricInhPa,950,20230201,300,1,cp,1,rotated_ll


We can convert the dataset to an xarray Dataset.

In [9]:
dataset = ds.to_xarray()
dataset

<xarray.Dataset>
Dimensions:        (number: 1, time: 1, step: 1, isobaricInhPa: 21, y: 786, x: 1170)
Coordinates:
  * number         (number) int64 1
  * time           (time) datetime64[ns] 2023-02-01T03:00:00
  * step           (step) timedelta64[ns] 01:00:00
  * isobaricInhPa  (isobaricInhPa) float64 1e+03 975.0 950.0 ... 150.0 100.0
    latitude       (y, x) float64 ...
    longitude      (y, x) float64 ...
    valid_time     (time, step) datetime64[ns] ...
Dimensions without coordinates: y, x
Data variables:
    t              (number, time, step, isobaricInhPa, y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             lssw
    GRIB_centreDescription:  Zurich
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Zurich
    history:                 2023-10-16T10:08 GRIB to CDM+CF via cfgrib-0.9.1...